# Evaluate military vehicle detections on The Search_2 dataset

This notebook provides a small benchmark to evaluate automated target recognition of military vehicles on a real dataset. Deep-learning object detection models can give reasonable detection performance when fine-tuned on specific datasets. However, acquiring enough data corresponding to a real military setting is a challenge, as demonstrated by this project. It is therefore important to evaluate these models in a military setting, with a target area around tens of pixels in a cluttered environment.

We propose to use [The Search_2](https://figshare.com/articles/dataset/The_Search_2_dataset/1041463) dataset for such an evaluation. The Search_2 dataset consists of 44 high-resolution digital color images of different complex natural scenes, with each scene (image) containing a single military vehicle that serves as a search target. Ground truth annotations are provided for the targets.

### Download and load the dataset

To begin, we download the dataset and load it into fiftyone.

In [ ]:
from orion.config.settings import settings
from orion.datasets.search2 import download, load_search_2_dataset

search_2_dir = settings.ORION_HOME_DIR / "search_2"
download(search_2_dir)
dataset = load_search_2_dataset(search_2_dir / "search_2")

We map the labels which identify each target in the dataset to the four classes with which our model was trained.

In [ ]:
from orion.datasets.search2 import LABEL_MAPPING

dataset.map_labels("ground_truth", LABEL_MAPPING).save()

### Evaluate a pretrained model on this dataset

Once our test dataset is ready, we can evaluate a pretrained model on it.

In [ ]:
from pathlib import Path

from orion.yolo.yolo import predict

model = Path.cwd().parent / "resources/models/orion12n.pt"
results_predict = predict(model, data=search_2_dir / "search_2/images")

# Load the path of the prediction model results
results_predict_dir = Path(results_predict[0].save_dir) # type: ignore

Let's load the model's predictions into our dataset.

In [ ]:
from orion.yolo.utils import add_yolo_detections

prediction_field = "yolo12"
predictions_dir = results_predict_dir / "labels"
add_yolo_detections(
    dataset,
    prediction_field=prediction_field,
    predictions_dir=predictions_dir,
    class_list=["AFV", "APC", "MEV", "LAV"],
)

Once that's done, we can evaluate our model's predictions and print the mean Average Precision (mAP).

In [ ]:
detection_results = dataset.evaluate_detections(
    prediction_field,
    eval_key="eval",
    compute_mAP=True,
    gt_field="ground_truth",
)

In [ ]:
mAP = detection_results.mAP()
print(f"mAP = {mAP}")

In [ ]:
detection_results.print_report()

We don't have a lot of test images, but our scores aren't good anyways... It's maybe easier to visualize the results in fiftyone.

In [ ]:
import fiftyone as fo

session = fo.launch_app(dataset, auto=False)
session.open_tab()